In [ ]:
# get rid of some chemprop warnings
# import warnings

# warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import os

import pandas as pd

# load the data
df = pd.read_csv('/home/s2861704/ppk/test_kin/prep_storage/K200_EGFR_TEST.csv')

display(df.head())

In [ ]:
# pivot the dataframe to have the accession as columns
df = df.pivot(index="SMILES", columns="accession", values="pchembl_value_Mean")
#df.columns.name = None
df.reset_index(inplace=True)
print(df.columns)

display(df.head())
#Teams message + delete smiles

In [ ]:
# print(len(df))
# max_Nans = len(df) - 30
# print(max_Nans)

In [ ]:
# df_noempty = df
# df_noempty = df_noempty.drop(list(df.columns[df.isna().sum() > max_Nans]), axis=1)
# df = df_noempty.copy()
# display(df)

In [ ]:
column_list = list(df.columns)
print (column_list)

In [ ]:
del column_list[0:1]
print(column_list)

In [ ]:
from qsprpred.data import QSPRDataset
from sklearn.impute import SimpleImputer

target_props = []
for target in column_list:
    target_props.append({'name': target, 'task': "REGRESSION"})

In [ ]:
print(target_props)

In [ ]:
targets_kinase200 = df.columns[1:7].tolist()
display(targets_kinase200)

In [ ]:
from qsprpred.data import QSPRDataset, RandomSplit
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data.descriptors.sets import SmilesDesc
from qsprpred.data.sampling.splits import GBMTRandomSplit, GBMTDataSplit
from split import random_global_equilibrated_random_split, dissimilaritydrive_global_balanced_cluster_split

df = dissimilaritydrive_global_balanced_cluster_split(data=df, targets=targets_kinase200, threads=8, sizes = [0.8,0.1,0.1])

In [ ]:
display(df)
print(df['split'].unique())
print(len(df[df['split']==0]))
print(len(df[df['split']=='train']))
print(len(df[df['split']=='test']))
df.to_csv(path_or_buf='/home/s2861704/ppk/test_kin/test_output/test_split.csv', sep=',', index=False)

In [1]:
#reload packages for df reload

import os
import pandas as pd
from qsprpred.data import QSPRDataset
from sklearn.impute import SimpleImputer
from qsprpred.data import QSPRDataset, RandomSplit
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data.descriptors.sets import SmilesDesc
from qsprpred.data.sampling.splits import GBMTRandomSplit, GBMTDataSplit
from split import random_global_equilibrated_random_split, dissimilaritydrive_global_balanced_cluster_split

In [4]:
df=pd.read_csv('/home/s2861704/ppk/test_kin/test_output/test_split.csv', sep=',')

column_list = list(df.columns[1:7])
# del column_list[0:1]
target_props = []
for target in column_list:
    target_props.append({'name': target, 'task': "REGRESSION"})
print(target_props)

[{'name': 'P00533', 'task': 'REGRESSION'}, {'name': 'P04626', 'task': 'REGRESSION'}, {'name': 'P17948', 'task': 'REGRESSION'}, {'name': 'P35916', 'task': 'REGRESSION'}, {'name': 'P35968', 'task': 'REGRESSION'}, {'name': 'Q15303', 'task': 'REGRESSION'}]


In [5]:
dataset = QSPRDataset(
    name="ChempropMTTestDataset",
    df=df,
    target_props=target_props,
    store_dir="/home/s2861704/ppk/test_kin/test_output/MT/data",
    random_state=42,
    drop_empty=False,
    overwrite=True
)

dataset.getDF()

,SMILES,P00533,P04626,P17948,P35916,P35968,Q15303,QSPRID,P00533_original,P04626_original,P17948_original,P35916_original,P35968_original,Q15303_original,split
QSPRID,,,,,,,,,,,,,,,
ChempropMTTestDataset_00000,Brc1cc2c(NCc3ccccc3)ncnc2s1,6.620,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00000,6.620,NaN,NaN,NaN,NaN,NaN,train
ChempropMTTestDataset_00001,Brc1cc2c(NCc3ccccn3)ncnc2s1,5.100,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00001,5.100,NaN,NaN,NaN,NaN,NaN,train
ChempropMTTestDataset_00002,Brc1cc2c(NCc3cccs3)ncnc2s1,5.860,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00002,5.860,NaN,NaN,NaN,NaN,NaN,train
ChempropMTTestDataset_00003,Brc1cc2c(NCc3ccncc3)ncnc2s1,5.410,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00003,5.410,NaN,NaN,NaN,NaN,NaN,train
ChempropMTTestDataset_00004,Brc1cc2c(Nc3ccccc3)ncnc2s1,7.100,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00004,7.100,NaN,NaN,NaN,NaN,NaN,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChempropMTTestDataset_13855,c1csc(-c2n[nH]c3c2Cc2ccccc2-3)c1,NaN,NaN,NaN,NaN,6.03,NaN,ChempropMTTestDataset_13855,NaN,NaN,NaN,NaN,6.03,NaN,test
ChempropMTTestDataset_13856,c1csc(-c2nnc(Cc3nc4ccccc4[nH]3)o2)c1,4.000,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_13856,4.000,NaN,NaN,NaN,NaN,NaN,test
ChempropMTTestDataset_13857,c1nc(Nc2ccc3[nH]ccc3c2)c2sc(-c3ccc(NCCN4CCCC4)...,9.000,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_13857,9.000,NaN,NaN,NaN,NaN,NaN,train


In [6]:

feature_calculators = [SmilesDesc(), MorganFP(radius=3, nBits=2048)]
dataset.prepareDataset(
    # split=random_global_equilibrated_random_split(data=df, targets=targets_kinase200, seed=2025),
    feature_calculators=feature_calculators,
    recalculate_features=True,
)

dataset.getDF().head()

,SMILES,P00533,P04626,P17948,P35916,P35968,Q15303,QSPRID,P00533_original,P04626_original,P17948_original,P35916_original,P35968_original,Q15303_original,split
QSPRID,,,,,,,,,,,,,,,
ChempropMTTestDataset_00000,Brc1cc2c(NCc3ccccc3)ncnc2s1,6.62,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00000,6.62,NaN,NaN,NaN,NaN,NaN,train
ChempropMTTestDataset_00001,Brc1cc2c(NCc3ccccn3)ncnc2s1,5.10,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00001,5.10,NaN,NaN,NaN,NaN,NaN,train
ChempropMTTestDataset_00002,Brc1cc2c(NCc3cccs3)ncnc2s1,5.86,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00002,5.86,NaN,NaN,NaN,NaN,NaN,train
ChempropMTTestDataset_00003,Brc1cc2c(NCc3ccncc3)ncnc2s1,5.41,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00003,5.41,NaN,NaN,NaN,NaN,NaN,train
ChempropMTTestDataset_00004,Brc1cc2c(Nc3ccccc3)ncnc2s1,7.10,NaN,NaN,NaN,NaN,NaN,ChempropMTTestDataset_00004,7.10,NaN,NaN,NaN,NaN,NaN,train


In [7]:
from qsprpred.extra.gpu.models.chemprop import ChempropModel

ChempropModel.getAvailableParameters()

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


{'no_cuda': 'Turn off cuda (i.e., use CPU instead of GPU).',
 'gpu': 'Which GPU to use.',
 'num_workers': 'Number of workers for the parallel data loading (0 means sequential).',
 'batch_size': 'Batch size.',
 'no_cache_mol': 'Whether to not cache the RDKit molecule for each SMILES string to reduce memory usage (cached by default).',
 'empty_cache': 'Whether to empty all caches before training or predicting. This is necessary if multiple jobs are run within a single script and the atom or bond features change.',
 'loss_function': 'Choice of loss function. Loss functions are limited to compatible dataset types.',
 'metric': "Metric to use with the validation set for early stopping. Defaults to 'auc' for classification, 'rmse' for regression. Note. In Chemprop this metric is also used for test-set evaluation, but in QSPRpred this is determined by the scoring parameter in assessment.",
 'bias': 'Whether to add bias to linear layers.',
 'hidden_size': 'Dimensionality of hidden layers in MP

In [8]:
# Create model
os.makedirs("/home/s2861704/ppk/test_kin/test_output/MT/models", exist_ok=True)
model = ChempropModel(
    base_dir='/home/s2861704/ppk/test_kin/test_output/MT/models',
    name='ChempropMTTestModel',
    parameters={"epochs": 5},
    quiet_logger=False
)

/home/s2861704/.conda/envs/drugex_env/lib/python3.12/site-packages/chemprop/utils.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(path, map_location

In [9]:
from qsprpred.models import SklearnMetrics, MaskedMetric
from qsprpred.plotting import regression
from qsprpred.tasks import ModelTasks
import numpy as np

r2_score=SklearnMetrics('r2')
masked_metric=MaskedMetric(metric=SklearnMetrics('r2'))

In [10]:
from qsprpred.models import CrossValAssessor, TestSetAssessor
from qsprpred.models.early_stopping import EarlyStoppingMode

# We can now assess the model performance on the training set using cross validation
CrossValAssessor(masked_metric, split_multitask_scores=True)(model, dataset)

# and on the test set
TestSetAssessor(masked_metric, split_multitask_scores=True)(model, dataset)

# Finally, we need to fit the model on the complete dataset if we want to use it further
# This will save the fitted model and metadata to disk
model.earlyStopping.numEpochs = 20
model.fitDataset(dataset, mode=EarlyStoppingMode.FIXED)

# We can optionally save the model and metadata to disk explicitly as well
_ = model.save()

Total size = 11,088
Fitting scaler
Number of parameters = 356,706
Moving trained model to cuda
  0%|          | 0/20 [00:00<?, ?it/s]Epoch 0
Loss = 1.0883e+00, PNorm = 34.0941, GNorm = 4.8391, lr_0 = 1.2240e-04
Loss = 1.0497e+00, PNorm = 34.0939, GNorm = 1.3881, lr_0 = 1.4276e-04
Loss = 1.0907e+00, PNorm = 34.0949, GNorm = 3.0281, lr_0 = 1.6312e-04
Loss = 1.0427e+00, PNorm = 34.0969, GNorm = 0.9390, lr_0 = 1.8348e-04
Loss = 1.0738e+00, PNorm = 34.1002, GNorm = 1.6998, lr_0 = 2.0385e-04
Loss = 9.7490e-01, PNorm = 34.1056, GNorm = 3.7674, lr_0 = 2.2421e-04
Loss = 9.5015e-01, PNorm = 34.1125, GNorm = 1.4132, lr_0 = 2.4457e-04
Loss = 1.0016e+00, PNorm = 34.1212, GNorm = 3.4615, lr_0 = 2.6493e-04
Loss = 8.0714e-01, PNorm = 34.1313, GNorm = 3.8059, lr_0 = 2.8529e-04
Loss = 1.0572e+00, PNorm = 34.1435, GNorm = 1.5919, lr_0 = 3.0566e-04
Loss = 1.0667e+00, PNorm = 34.1533, GNorm = 3.6540, lr_0 = 3.2602e-04
Loss = 9.1792e-01, PNorm = 34.1667, GNorm = 1.3565, lr_0 = 3.4638e-04
Loss = 9.1159e-01, 

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
import pandas as pd
from sklearn.metrics import explained_variance_score, mean_absolute_error, \
    mean_squared_error, r2_score
import math

# get independent test set
df = pd.read_table(
    "/home/s2861704/ppk/test_kin/test_output/MT/models/ChempropMTTestModel/ChempropMTTestModel.ind.tsv")

# filling NaN-values with fillna() makes the code work, but results in the same issues imputation has. next step to try: individual target plotting
# df = df.fillna(0)

# column names containing original labels or predictions for the tasks
label_names = [i for i in list(df.columns.values) if "Label" in i]
pred_names = [i for i in list(df.columns.values) if "Prediction" in i]

# turn into np array
ylabel = df[label_names].to_numpy()
ypred = df[pred_names].to_numpy()

# get metrics
summary = {}
for metric in [explained_variance_score, mean_absolute_error, mean_squared_error,
               r2_score]:
    score = metric(ylabel, ypred)
    summary[metric.__name__] = score

summary["ModelName"] = model.name

summary

In [ ]:
from plotting_regression_MT import CorrelationPlot

plot = CorrelationPlot([model])
axes, summary = plot.make(save=True, show=True, out_path='/home/s2861704/ppk/test_kin/test_output/ChempropMT.png')

In [ ]:
summary